In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import urllib
import datetime as dt
import pandas as pd

## Functions

In [37]:
def soupifyWeb(url):
    # Takes url. Parces page and Returns allTeam's info (name and url to youtube)
    try:
        #load website & parse with bs4
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page,'html.parser')
    except URLError as e:
        print(e)
        
    # get likes with unclicked tag
    try:
        print('finding all divs...')
        soup = soup.find_all('div', class_="fl-post-column")
        print('All teams found = ', len(soup))
        return soup
    except HTTPError as e:
        print(e)

In [36]:
def extractInfo(aSoup):
    teamInfo = []
    for i in aSoup:
        try:
            youtubeUrl = i.find_all('iframe')[0].attrs['src']
            #ignore youku urls 
            #TODO: remove this after figuring how to scrappe youku likes 
            if 'youku'in youtubeUrl:
                pass
            else:
                youtubeUrl = youtubeUrl.replace('embed/', 'watch?v=') # Fix URL from "embeded"  to "watch"
                teamName = i.find_all('h5', class_="uppercase post-team-name")[0].text
                teamInfo.append([teamName, youtubeUrl])
        except:
            print("no src or h5 found")
    print('Teams found (youtube Only) : ', len(teamInfo))
    return teamInfo

## Load and parce  eoshackathon.io

In [23]:
soup = soupifyWeb('https://eoshackathon.io/community-voting/#application')

finding all divs...
Soup len =  72


## Extract team names and youtube urls

In [35]:
teamsInfo = pd.DataFrame(extractInfo(soup))
teamsInfo.columns = ['team','url']

Teams found :  69


##  Save to csv

In [26]:
teamsInfo.to_csv("teamsInfo.csv", header=True, index=False)

# ----------- ADD new teams directly to  tracker.csv -------------

####  Load tracker

In [8]:
tracker = pd.read_csv("videoLikesTracker.csv")

####  check teams on teamsInfo not existent in tracker

In [15]:
newTeams = teamsInfo.loc[~teamsInfo['team'].isin(tracker['team'])]

In [16]:
newTeams.to_csv("newTeams.csv", header=True, index=False)

In [ ]:
tracker = tracker.set_index('team')
teamsInfo = teamsInfo.set_index('team')

In [ ]:
len(teamsInfo)

In [ ]:
len(tracker)

In [ ]:
tracker.append(teamsInfo)


#  Beautifullized html XD ------------

In [ ]:
pg = urllib.request.urlopen('https://eoshackathon.io/community-voting/#application')
sp = BeautifulSoup(pg,'html.parser')
print(sp.prettify())